In [1]:
import sys
sys.path.append('../')

import meta_dataloader.TCGA

import models.mlp, models.gcn
import numpy as np
import data.gene_graphs
import collections
import sklearn.metrics
import sklearn.model_selection

%load_ext autoreload
%autoreload 2

In [2]:
# tasks = meta_dataloader.TCGA.TCGAMeta(download=True, min_samples_per_class=10)
# task = tasks[113]

In [3]:
# for i, task in enumerate(tasks):
#     print (i, task.id, collections.Counter(task._labels))

In [4]:
#task = meta_dataloader.TCGA.TCGATask(('PAM50Call_RNAseq', 'BRCA'))
task = meta_dataloader.TCGA.TCGATask(('gender', 'GBM'))


In [5]:
print(task.id)
print(task._samples.shape)
print(np.asarray(task._labels).shape)
print(collections.Counter(task._labels))

('gender', 'GBM')
(166, 16300)
(166,)
Counter({1: 107, 0: 59})


In [6]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(task._samples, 
                                                                            task._labels, 
                                                                            stratify=task._labels,
                                                                            train_size=50,
                                                                            test_size=100,
                                                                            shuffle=True,
                                                                            random_state=0
                                                                             )
X_test, X_valid, y_test, y_valid = sklearn.model_selection.train_test_split(X_test, 
                                                                            y_test, 
                                                                            stratify=y_test,
                                                                            train_size=50,
                                                                            test_size=50,
                                                                            shuffle=True,
                                                                            random_state=0
                                                                           )

In [34]:
for lr in [0.0001,0.001,0.01,0.1]:
    model = models.mlp.MLP(name="MLP_lay2_chan512",
                           num_layer=1, 
                           channels=256, 
                           lr=lr,
                           patience=50,
                           cuda=True,
                           metric=sklearn.metrics.accuracy_score,
                           verbose=False)

    model.fit(X_train, y_train)

    y_valid_pred = model.predict(X_valid)
    print(lr, sklearn.metrics.accuracy_score(y_valid, np.argmax(y_valid_pred,axis=1)))


0.0001 0.7
0.001 0.72
0.01 0.64
0.1 0.64


In [8]:
# y_pred = model.predict(X_test)
# y_pred = np.argmax(y_pred,axis=1)
# print(sklearn.metrics.accuracy_score(y_test, y_pred))


In [9]:
graph = data.gene_graphs.GeneManiaGraph()
adj = graph.adj()

Torrent name: genemania.pkl, Size: 9.61MB


In [21]:
import gc
gc.collect()

2011

In [26]:
for nl in [1]:
    for lr in [0.0001]:
        model = models.gcn.GCN(name="GCN_lay3_chan64_emb32_dropout_agg_hierarchy", 
                               dropout=False, 
                               cuda=True,
                               num_layer=nl,
                               prepool_extralayers=2,
                               channels=64, 
                               embedding=32, 
                               aggregation="hierarchy",
                               lr=lr,
                               num_epochs=200,
                               patience=100,
                               verbose=True
                              )
        model.fit(X_train, y_train, adj)

        y_valid_pred = model.predict(X_valid)
        print("###",lr,sklearn.metrics.accuracy_score(y_valid, np.argmax(y_valid_pred,axis=1)))


Early stopping metric is accuracy_score


/network/home/cohenjos/.local/lib/python3.6/site-packages/scipy/sparse/compressed.py:708: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self[i, j] = values


Reducing graph by a factor of 2 to 8150 nodes
Found cache for /network/tmp1/cohenjos/workspace/gene-graph-conv/.cache/hierarchicalb9a8a435080edb44dd7545b40248ef139ecffef54a4be93b3a89847e040373058150.npy
  batch (0/40), train loss:0.6872
  batch (10/40), train loss:2.5687
  batch (20/40), train loss:2.0576
  batch (30/40), train loss:0.8775
epoch: 0, time: 0.40, valid_metric: 0.40, train_metric: 0.35
  batch (0/40), train loss:1.7699
  batch (10/40), train loss:1.3001
  batch (20/40), train loss:0.7544
  batch (30/40), train loss:0.7951
epoch: 1, time: 0.40, valid_metric: 0.60, train_metric: 0.65
  batch (0/40), train loss:0.5796
  batch (10/40), train loss:1.2767
  batch (20/40), train loss:1.1345
  batch (30/40), train loss:0.8006
epoch: 2, time: 0.40, valid_metric: 0.40, train_metric: 0.35
  batch (0/40), train loss:0.6981
  batch (10/40), train loss:0.8128
  batch (20/40), train loss:0.8097
  batch (30/40), train loss:0.6966
epoch: 3, time: 0.40, valid_metric: 0.60, train_metric: 0.

  batch (10/40), train loss:0.4368
  batch (20/40), train loss:0.5243
  batch (30/40), train loss:0.4656
epoch: 40, time: 0.40, valid_metric: 0.70, train_metric: 0.97
  batch (0/40), train loss:0.4772
  batch (10/40), train loss:0.4240
  batch (20/40), train loss:0.5138
  batch (30/40), train loss:0.4546
epoch: 41, time: 0.40, valid_metric: 0.70, train_metric: 0.97
  batch (0/40), train loss:0.4680
  batch (10/40), train loss:0.4111
  batch (20/40), train loss:0.5029
  batch (30/40), train loss:0.4434
epoch: 42, time: 0.40, valid_metric: 0.70, train_metric: 0.97
  batch (0/40), train loss:0.4590
  batch (10/40), train loss:0.3982
  batch (20/40), train loss:0.4918
  batch (30/40), train loss:0.4322
epoch: 43, time: 0.40, valid_metric: 0.70, train_metric: 0.97
  batch (0/40), train loss:0.4496
  batch (10/40), train loss:0.3851
  batch (20/40), train loss:0.4802
  batch (30/40), train loss:0.4208
epoch: 44, time: 0.40, valid_metric: 0.70, train_metric: 0.97
  batch (0/40), train loss:0.

  batch (10/40), train loss:0.0472
  batch (20/40), train loss:0.0822
  batch (30/40), train loss:0.0774
epoch: 81, time: 0.40, valid_metric: 0.70, train_metric: 1.00
  batch (0/40), train loss:0.0766
  batch (10/40), train loss:0.0441
  batch (20/40), train loss:0.0774
  batch (30/40), train loss:0.0728
epoch: 82, time: 0.40, valid_metric: 0.70, train_metric: 1.00
  batch (0/40), train loss:0.0722
  batch (10/40), train loss:0.0413
  batch (20/40), train loss:0.0729
  batch (30/40), train loss:0.0685
epoch: 83, time: 0.40, valid_metric: 0.70, train_metric: 1.00
  batch (0/40), train loss:0.0682
  batch (10/40), train loss:0.0386
  batch (20/40), train loss:0.0687
  batch (30/40), train loss:0.0644
epoch: 84, time: 0.40, valid_metric: 0.70, train_metric: 1.00
  batch (0/40), train loss:0.0644
  batch (10/40), train loss:0.0362
  batch (20/40), train loss:0.0647
  batch (30/40), train loss:0.0606
epoch: 85, time: 0.40, valid_metric: 0.70, train_metric: 1.00
  batch (0/40), train loss:0.

  batch (10/40), train loss:0.0058
  batch (20/40), train loss:0.0115
  batch (30/40), train loss:0.0099
epoch: 122, time: 0.40, valid_metric: 0.80, train_metric: 1.00
  batch (0/40), train loss:0.0112
  batch (10/40), train loss:0.0056
  batch (20/40), train loss:0.0111
  batch (30/40), train loss:0.0096
epoch: 123, time: 0.40, valid_metric: 0.80, train_metric: 1.00
  batch (0/40), train loss:0.0109
  batch (10/40), train loss:0.0054
  batch (20/40), train loss:0.0108
  batch (30/40), train loss:0.0092
epoch: 124, time: 0.40, valid_metric: 0.80, train_metric: 1.00
  batch (0/40), train loss:0.0105
  batch (10/40), train loss:0.0052
  batch (20/40), train loss:0.0104
  batch (30/40), train loss:0.0089
epoch: 125, time: 0.40, valid_metric: 0.80, train_metric: 1.00
  batch (0/40), train loss:0.0101
  batch (10/40), train loss:0.0051
  batch (20/40), train loss:0.0101
  batch (30/40), train loss:0.0086
epoch: 126, time: 0.40, valid_metric: 0.80, train_metric: 1.00
  batch (0/40), train lo

  batch (10/40), train loss:0.0018
  batch (20/40), train loss:0.0036
  batch (30/40), train loss:0.0030
epoch: 163, time: 0.40, valid_metric: 0.80, train_metric: 1.00
  batch (0/40), train loss:0.0035
  batch (10/40), train loss:0.0018
  batch (20/40), train loss:0.0035
  batch (30/40), train loss:0.0029
epoch: 164, time: 0.40, valid_metric: 0.80, train_metric: 1.00
  batch (0/40), train loss:0.0034
  batch (10/40), train loss:0.0017
  batch (20/40), train loss:0.0034
  batch (30/40), train loss:0.0029
epoch: 165, time: 0.40, valid_metric: 0.80, train_metric: 1.00
  batch (0/40), train loss:0.0033
  batch (10/40), train loss:0.0017
  batch (20/40), train loss:0.0033
  batch (30/40), train loss:0.0028
epoch: 166, time: 0.40, valid_metric: 0.80, train_metric: 1.00
  batch (0/40), train loss:0.0033
  batch (10/40), train loss:0.0017
  batch (20/40), train loss:0.0033
  batch (30/40), train loss:0.0027
epoch: 167, time: 0.40, valid_metric: 0.80, train_metric: 1.00
  batch (0/40), train lo

In [27]:
y_pred = model.predict(X_test)
print(y_pred)
y_pred = np.argmax(y_pred,axis=1)
print(y_pred)
print(y_test)
print(sklearn.metrics.accuracy_score(y_test, y_pred))

tensor([[6.2505e-01, 3.7495e-01],
        [1.1698e-03, 9.9883e-01],
        [1.0086e-01, 8.9914e-01],
        [4.0226e-04, 9.9960e-01],
        [7.4587e-01, 2.5413e-01],
        [2.6804e-02, 9.7320e-01],
        [8.2272e-02, 9.1773e-01],
        [1.6810e-03, 9.9832e-01],
        [2.2646e-03, 9.9774e-01],
        [4.9212e-03, 9.9508e-01],
        [4.3442e-04, 9.9957e-01],
        [6.2665e-06, 9.9999e-01],
        [2.6350e-02, 9.7365e-01],
        [6.0380e-02, 9.3962e-01],
        [6.3009e-01, 3.6991e-01],
        [6.2994e-02, 9.3701e-01],
        [9.2387e-01, 7.6130e-02],
        [1.3427e-04, 9.9987e-01],
        [6.9219e-02, 9.3078e-01],
        [5.4604e-01, 4.5396e-01],
        [4.1000e-01, 5.9000e-01],
        [5.1533e-03, 9.9485e-01],
        [9.5795e-01, 4.2046e-02],
        [8.2147e-02, 9.1785e-01],
        [2.9282e-01, 7.0718e-01],
        [2.3493e-04, 9.9977e-01],
        [5.3314e-03, 9.9467e-01],
        [1.9394e-03, 9.9806e-01],
        [1.1794e-01, 8.8206e-01],
        [4.193